In [5]:
--pierwsza tabela. (test, czy w ogóle działa)
CREATE TABLE Studenci ( id_student char(10) NOT NULL PRIMARY KEY, imie varchar(20) NOT NULL, nazwisko varchar(20) NOT NULL, hash_haslo varchar(256));

Commands completed successfully.

Total execution time: 00:00:00.047

In [6]:
--tworzenie pozostałych tabel;
CREATE TABLE Wydzialy (wydzial varchar(100) NOT NULL PRIMARY KEY);
CREATE TABLE Rodzaje (rodzaj varchar(50) NOT NULL PRIMARY KEY);
CREATE TABLE Stopnie (stopien varchar(30) NOT NULL PRIMARY KEY);
CREATE TABLE Studia ( id_studia int NOT NULL IDENTITY(1,1) PRIMARY KEY, wydzial varchar(100) NOT NULL FOREIGN KEY REFERENCES Wydzialy(wydzial), kierunek varchar(100) NOT NULL, rodzaj varchar(50) NOT NULL FOREIGN KEY REFERENCES Rodzaje(rodzaj) );
CREATE TABLE Pracownicy ( id_pracownik int NOT NULL IDENTITY(1,1) PRIMARY KEY, imie varchar(20) NOT NULL, nazwisko varchar(20) NOT NULL, stopien varchar(30) FOREIGN KEY REFERENCES Stopnie(stopien), czyPodgladacz bit DEFAULT 0, czyHistoryk bit DEFAULT 0, czyWprowadzaczSystemu bit DEFAULT 0, czyAktualizator bit DEFAULT 0, czyZarzadzajacy bit DEFAULT 0, hash_haslo varchar(256));
CREATE TABLE Prace ( id_praca int NOT NULL IDENTITY(1,1) PRIMARY KEY, tytul varchar(100) NOT NULL UNIQUE, dataDodania datetime DEFAULT GETDATE(), id_promotor int DEFAULT 1 FOREIGN KEY REFERENCES Pracownicy(id_pracownik), ocenaPromotora decimal(2,1) CHECK (ocenaPromotora >= 2.0 AND ocenaPromotora <= 5.0), ocenaKoncowa decimal(2,1) CHECK (ocenaKoncowa >= 2.0 AND ocenaKoncowa <= 5.0),  dataObrony datetime, liczbaRobotnikow tinyint DEFAULT 1 CHECK (liczbaRobotnikow >= 1 AND liczbaRobotnikow <= 3), wydzial varchar(100) NOT NULL FOREIGN KEY REFERENCES Wydzialy(wydzial), id_szefObrony int DEFAULT 1 FOREIGN KEY REFERENCES Pracownicy(id_pracownik), grupaRodzaj varchar(50) NOT NULL FOREIGN KEY REFERENCES Rodzaje(rodzaj));
CREATE TABLE StudenciStudia ( id_student char(10) NOT NULL FOREIGN KEY REFERENCES Studenci(id_student), id_studia int NOT NULL FOREIGN KEY REFERENCES Studia(id_studia), id_student_studia int NOT NULL IDENTITY(2,1) PRIMARY KEY );
CREATE TABLE StudenciPrace ( id_student char(10) NOT NULL FOREIGN KEY REFERENCES Studenci(id_student), id_praca int NOT NULL FOREIGN KEY REFERENCES Prace(id_praca), ocena decimal(2,1) CHECK (ocena >= 2.0 AND ocena <= 5.0), id_student_studia int NOT NULL FOREIGN KEY REFERENCES StudenciStudia(id_student_studia), PRIMARY KEY (id_student, id_praca));
CREATE TABLE Slowa ( slowo varchar(100) NOT NULL PRIMARY KEY);
CREATE TABLE PraceSlowa ( id_praca int NOT NULL FOREIGN KEY REFERENCES Prace(id_praca), slowo varchar(100) NOT NULL FOREIGN KEY REFERENCES Slowa(slowo), PRIMARY KEY (id_praca, slowo));
CREATE TABLE PraceRecenzenci ( id_praca int NOT NULL FOREIGN KEY REFERENCES Prace(id_praca), id_recenzent int NOT NULL FOREIGN KEY REFERENCES Pracownicy(id_pracownik), ocena decimal(2,1) CHECK (ocena >= 2.0 AND ocena <= 5.0), PRIMARY KEY (id_praca, id_recenzent));

Commands completed successfully.

Total execution time: 00:00:00.125

In [7]:
--Wyzwalacze.
--wyzwalacz wymuszający konkretny format danych tabeli Wydzialy.
CREATE TRIGGER trig_ai_Wydzialy ON Wydzialy AFTER INSERT AS BEGIN UPDATE Wydzialy SET Wydzialy.wydzial = UPPER(TRIM(Wydzialy.wydzial)) FROM inserted WHERE Wydzialy.wydzial = inserted.wydzial; END;

Commands completed successfully.

Total execution time: 00:00:00.047

In [8]:
--podobne wyzwalacze dla innych tabel
CREATE TRIGGER trig_ai_Studia ON Studia AFTER INSERT AS BEGIN UPDATE Studia SET Studia.kierunek = UPPER(TRIM(Studia.kierunek)) FROM inserted WHERE Studia.kierunek = inserted.kierunek; END;

Commands completed successfully.

Total execution time: 00:00:00.067

In [9]:
CREATE TRIGGER trig_ai_Rodzaje ON Rodzaje AFTER INSERT AS BEGIN UPDATE Rodzaje SET Rodzaje.rodzaj = UPPER(TRIM(Rodzaje.rodzaj)) FROM inserted WHERE Rodzaje.rodzaj = inserted.rodzaj; END;

Commands completed successfully.

Total execution time: 00:00:00.045

In [10]:
CREATE TRIGGER trig_ai_Studenci ON Studenci AFTER INSERT AS BEGIN UPDATE Studenci SET Studenci.id_student = TRIM(Studenci.id_student) FROM inserted WHERE Studenci.id_student = inserted.id_student; END;

Commands completed successfully.

Total execution time: 00:00:00.048

In [11]:
CREATE TRIGGER trig_ai_Stopnie ON Stopnie AFTER INSERT AS BEGIN UPDATE Stopnie SET Stopnie.stopien = UPPER(TRIM(Stopnie.stopien)) FROM inserted WHERE Stopnie.stopien = inserted.stopien; END;

Commands completed successfully.

Total execution time: 00:00:00.044

In [12]:
--analogiczne wyzwalacze dla aktualizacji danych w tych tabelach
CREATE TRIGGER trig_au_Wydzialy ON Wydzialy AFTER UPDATE AS BEGIN UPDATE Wydzialy SET Wydzialy.wydzial = UPPER(TRIM(Wydzialy.wydzial)) FROM inserted WHERE Wydzialy.wydzial = inserted.wydzial; END
GO
CREATE TRIGGER trig_au_Studia ON Studia AFTER UPDATE AS BEGIN UPDATE Studia SET Studia.kierunek = UPPER(TRIM(Studia.kierunek)) FROM inserted WHERE Studia.kierunek = inserted.kierunek; END
GO
CREATE TRIGGER trig_au_Rodzaje ON Rodzaje AFTER UPDATE AS BEGIN UPDATE Rodzaje SET Rodzaje.rodzaj = UPPER(TRIM(Rodzaje.rodzaj)) FROM inserted WHERE Rodzaje.rodzaj = inserted.rodzaj; END
GO
CREATE TRIGGER trig_au_Studenci ON Studenci AFTER UPDATE AS BEGIN UPDATE Studenci SET Studenci.id_student = TRIM(Studenci.id_student) FROM inserted WHERE Studenci.id_student = inserted.id_student; END
GO
CREATE TRIGGER trig_au_Stopnie ON Stopnie AFTER UPDATE AS BEGIN UPDATE Stopnie SET Stopnie.stopien = UPPER(TRIM(Stopnie.stopien)) FROM inserted WHERE Stopnie.stopien = inserted.stopien; END

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.228

In [13]:
--wyzwlacz zapewniający odpowiedniego promotora
CREATE TRIGGER trig_biu_Prace ON Prace AFTER INSERT, UPDATE AS 
BEGIN
DECLARE @stopref varchar(30);
DECLARE @insertedidpromotor int;
SELECT @insertedidpromotor = id_promotor FROM inserted;
SELECT @stopref = stopien FROM Pracownicy WHERE Pracownicy.id_pracownik = @insertedidpromotor;
IF ((@stopref NOT LIKE '%DR%') AND (@stopref NOT LIKE '%PROF%')) 
ROLLBACK TRANSACTION;
END

Commands completed successfully.

Total execution time: 00:00:00.045

In [14]:
--wyzwalacz zapewniający, że recenzent nie będzie promotorem tej samej pracy
CREATE TRIGGER trig_biu_PraceRecenzenci ON PraceRecenzenci AFTER INSERT, UPDATE AS
BEGIN
DECLARE @refprom int;
DECLARE @insertedpraca int;
DECLARE @insertedrecenzent int;
SELECT @insertedpraca = id_praca, @insertedrecenzent = id_recenzent FROM inserted;
SELECT @refprom = id_promotor FROM Prace WHERE id_praca = @insertedpraca;
IF @insertedrecenzent = @refprom
ROLLBACK TRANSACTION;
END

Commands completed successfully.

Total execution time: 00:00:00.053

In [15]:
--wyzwalacz zapewniający, że każda praca ma przynajmniej jedno słowo kluczowe
CREATE TRIGGER trig_ai_Prace ON Prace AFTER INSERT AS
BEGIN
DECLARE @insertedidpraca int;
SELECT @insertedidpraca = id_praca FROM inserted;
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (@insertedidpraca, 'PRACA DYPLOMOWA');
END

Commands completed successfully.

Total execution time: 00:00:00.044

In [16]:
--procedura do następnego wyzwalacza sprawdzającego czy podany student w ogóle może dodać odpowiednią pracę
CREATE PROCEDURE sp_trig_ai_StudenciPrace @idstudent CHAR(10), @idpraca INT, @idstudentstudia INT, @info bit OUT
AS
BEGIN 
DECLARE @wydzialWprace VARCHAR(100); 
DECLARE @wydzialWstudia VARCHAR(100); 
DECLARE @rodzajWprace VARCHAR(50); 
DECLARE @rodzajWstudia VARCHAR(50); 
DECLARE @gotowe BIT; 
SET @info = 0;
SELECT @wydzialWprace = Prace.wydzial, @rodzajWprace = Prace.grupaRodzaj FROM Prace WHERE Prace.id_praca = @idpraca; 
SELECT @wydzialWstudia = Studia.wydzial, @rodzajWstudia = Studia.rodzaj FROM StudenciStudia INNER JOIN Studia ON StudenciStudia.id_studia = Studia.id_studia WHERE (StudenciStudia.id_student_studia = @idstudentstudia AND StudenciStudia.id_student = @idstudent);
IF ((@wydzialWstudia = @wydzialWprace) AND (@rodzajWstudia = @rodzajWprace))
    SET @info = 1; 
END;

Commands completed successfully.

Total execution time: 00:00:00.044

In [17]:
--wspomniany powyżej wyzwalacz czy podany student w ogóle może dodać odpowiednią pracę
CREATE TRIGGER trig_aiu_StudenciPrace ON StudenciPrace AFTER INSERT, UPDATE AS 
BEGIN
DECLARE @info bit;
DECLARE @insertedidstudent int;
DECLARE @insertedidpraca int;
DECLARE @insertedidstudentstudia int;
SET @info = 0; 
SELECT @insertedidstudent = id_student FROM inserted;
SELECT @insertedidpraca = id_praca FROM inserted;
SELECT @insertedidstudentstudia = id_student_studia FROM inserted;
EXEC sp_trig_ai_StudenciPrace @insertedidstudent, @insertedidpraca, @insertedidstudentstudia, @info OUT;
IF @info != 1 
ROLLBACK TRANSACTION; 
END;

Commands completed successfully.

Total execution time: 00:00:00.047

In [18]:
--dodawanie danych, cz.I. 
INSERT INTO Wydzialy (wydzial) VALUES ('elektryczny');
INSERT INTO Wydzialy (wydzial) VALUES ('matematyczno-przyrodniczy');
INSERT INTO Wydzialy (wydzial) VALUES ('nauk o zdrowiu');
INSERT INTO Wydzialy (wydzial) VALUES ('fizyki');
INSERT INTO Wydzialy (wydzial) VALUES ('ekonomii i administracji');
INSERT INTO Wydzialy (wydzial) VALUES ('chemii');
INSERT INTO Rodzaje (rodzaj) VALUES ('inżynierskie');
INSERT INTO Rodzaje (rodzaj) VALUES ('magisterskie');
INSERT INTO Rodzaje (rodzaj) VALUES ('licencjackie');
INSERT INTO Rodzaje (rodzaj) VALUES ('doktoranckie');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('ELEKTRYCZNY','informatyka stosowana','INŻYNIERSKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('ELEKTRYCZNY','elektrotechnika i elektronika','INŻYNIERSKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('EKONOMII I ADMINISTRACJI','ekonomia stosowana','LICENCJACKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('EKONOMII I ADMINISTRACJI','zarządzanie','LICENCJACKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('FIZYKI','fizyka stosowana','LICENCJACKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('FIZYKI','fizyka kwantowa i relatywistyczna','DOKTORANCKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('CHEMII','chemia organiczna','MAGISTERSKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('CHEMII','chemia mistrzów','DOKTORANCKIE');
INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES ('NAUK O ZDROWIU','pielęgniarstwo','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FIZYKI','Siła grawitacji przy budowie mostów','INŻYNIERSKIE');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('135','Anna','Andrzejewska');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('138','Bartosz','Benedyktyński');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('256','Czesława','Cymbalińska');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('413','Dionizy','Dogłębny');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('1123','Eleonora','Elektryfikująca');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('1563','Franciszek','Frankowski');
INSERT INTO Studenci (id_student, imie, nazwisko) VALUES ('35672','Franciszek','Ferdynancki');
INSERT INTO Studenci (id_student, imie, nazwisko) VALUES ('8747','Grzegorz','Gorycki');
INSERT INTO Studenci (id_student, imie, nazwisko) VALUES ('11234','Henryk','Heniawski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('987','Ilona','Iłanicka');
INSERT INTO Stopnie VALUES ('inż');
INSERT INTO Stopnie VALUES ('lic');
INSERT INTO Stopnie VALUES ('dr');
INSERT INTO Stopnie VALUES ('dr inż');
INSERT INTO Stopnie VALUES ('dr hab');
INSERT INTO Stopnie VALUES ('prof');
INSERT INTO Stopnie VALUES ('prof dr hab');
INSERT INTO Stopnie VALUES ('mgr');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('probny','probny','DR');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Rakoczy','Rondecki','PROF');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Zygfryd','Zebrzański','DR');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Paulina','Penderecka','DR HAB');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Tadeusz','Tokarski','MGR');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Sonia','Szarecka','LIC');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Ola','Oskarska','INŻ');
INSERT INTO Pracownicy (imie, nazwisko, stopien) VALUES ('Marek','Moczydłowski','DR INŻ');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('Marek', 'Moczydłowski', 'PROF');
INSERT INTO Slowa VALUES ('potencjał błonowy');
INSERT INTO Slowa VALUES ('nitrozaminy');
INSERT INTO Slowa VALUES ('neuroleptyki');
INSERT INTO Slowa VALUES ('prąd elektryczny');
INSERT INTO Slowa VALUES ('konstrukcje żelbetowe');
INSERT INTO Slowa VALUES ('nosorożce na sawannie');
INSERT INTO Slowa VALUES ('praca dyplomowa');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.969

In [19]:
--wprowadzanie danych cd.
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('1123',3);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('1123',9)
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('11234',6);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('11234',10);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('135',2);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('135',8);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('138',1);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('138',7);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('1563',6);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('256',5);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('35672',9);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('413',4);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('8747',9);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('987',7);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.120

In [20]:
--wstawianie danych cd.
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Nitrozaminy w komórkach ludzkich',2,3.0,'2020-03-12',1,'CHEMII',8,'MAGISTERSKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Opieka nad pacjentami z chorobą Crohna',3,3.0,'2021-03-12',3,'NAUK O ZDROWIU',7,'LICENCJACKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Kwantowa teoria pola w czarnych dziurach',2,4.0,'2019-06-11',2,'FIZYKI',6,'DOKTORANCKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Siły odśrodkowe w autobusach',4,5.0,'2021-06-21',1,'FIZYKI',5,'LICENCJACKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Inflacja w krajach Europy Środkowo-Wschodniej',8,4.5,'2018-12-21',1,'EKONOMII I ADMINISTRACJI',4,'LICENCJACKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Pomiary napięcia w procesorach x86',2,3.5,'2020-01-10',1,'ELEKTRYCZNY',3,'INŻYNIERSKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Zastosowania kryptografii na stronach rządowych',8,3.0,'2020-01-10',1,'ELEKTRYCZNY',2,'INŻYNIERSKIE');
INSERT INTO Prace (tytul,id_promotor,ocenaPromotora,dataObrony,liczbaRobotnikow,wydzial,id_szefObrony,grupaRodzaj) VALUES ('Elektrochemia kardiomiocytów',9,5.0,'2020-02-12',1,'CHEMII',3,'MAGISTERSKIE');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.120

In [21]:
UPDATE Prace SET ocenaKoncowa = 3.0;

(8 rows affected)

Total execution time: 00:00:00.050

In [22]:
--wstawianie danych cd.
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('138',1,4.0,9);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('1123',2,3.5,3);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('35672',2,3.5,12);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('1563',3,3.5,10);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('11234',3,3.5,4);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('8747',2,4.5,14);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('256',4,3.5,11);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('413',5,3.5,13);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('138',6,3.5,8);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('135',7,5.0,6);
INSERT INTO StudenciPrace (id_student, id_praca, ocena, id_student_studia) VALUES ('987',8,4.0,15);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.193

In [23]:
--wstawianie danych cd.
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (1,3,5.0);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (2,2,4.0);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (2,4,3.0);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (3,4,5.0);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (4,2,5.0);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (4,5,4.5);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (4,7,3.5);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (5,2,3.5);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (6,5,4.5);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (7,5,3.5);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.163

In [24]:
--wstawianie danych cd.
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (8,6,3.5);
INSERT INTO PraceRecenzenci (id_praca,id_recenzent,ocena) VALUES (8,8,4.5);
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (1,'KONSTRUKCJE ŻELBETOWE');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (1,'NITROZAMINY');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (2,'NITROZAMINY');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (3,'NEUROLEPTYKI');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (4,'NOSOROŻCE NA SAWANNIE');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (5,'POTENCJAŁ BŁONOWY');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (5,'NOSOROŻCE NA SAWANNIE');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (6,'PRĄD ELEKTRYCZNY');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (7,'PRĄD ELEKTRYCZNY');
INSERT INTO PraceSlowa (id_praca, slowo) VALUES (8,'PRĄD ELEKTRYCZNY');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.105

In [25]:
--uzupełnianie danych
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('58321','Jan','Janowski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583212','Karol','Kowalski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583213','Lucjan','Lubelski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583214','Łukasz','Łopiański');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583215','Mateusz','Manowski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583216','Nikodem','Nieznany');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583217','Olek','Olechowski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583218','Piotr','Pacanowski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('583219','Radosłwa','Rzeszowski');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('5832110','Stanisław','Słomiański');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('5832111','Tadeusz','Tomiański');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('5832112','Urszula','Uramowska');
INSERT INTO Studenci (id_student,imie,nazwisko) VALUES ('5832113','Wiesława','Witkowska');
INSERT INTO Wydzialy VALUES ('LEKARSKI1');
INSERT INTO Wydzialy VALUES ('LEKARSKI2');
INSERT INTO Wydzialy VALUES ('LEKARSKI3');
INSERT INTO Wydzialy VALUES ('MATEMATYCZNY2');
INSERT INTO Wydzialy VALUES ('MATEMATYCZNY3');
INSERT INTO Wydzialy VALUES ('MATEMATYCZNY4');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY1');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY2');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY3');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY4');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY5');
INSERT INTO Wydzialy VALUES ('HUMANISTYCZNY6');
INSERT INTO Wydzialy VALUES ('FARMACUTYCZNY1');
INSERT INTO Wydzialy VALUES ('FARMACEUTYCZNY2');
INSERT INTO Wydzialy VALUES ('FARMACEUTYCZNY3');
INSERT INTO Wydzialy VALUES ('FARMACEUTYCZNY4');
INSERT INTO Wydzialy VALUES ('FARMACEUTYCZNY5');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI1','fajnykierunek1','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI2','fajnykierunek2','MAGISTERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI3','fajnykierunek3','MAGISTERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI2','fajnykierunek4','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI3','fajnykierunek5','MAGISTERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('LEKARSKI1','fajnykierunek6','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY1','fajnykierunek7','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY2','fajnykierunek8','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY3','fajnykierunek9','INŻYNIERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY4','fajnykierunek10','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY5','fajnykierunek11','INŻYNIERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY6','fajnykierunek12','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('HUMANISTYCZNY1','fajnykierunek13','MAGISTERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACUTYCZNY1','fajnykierunek14','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACEUTYCZNY2','fajnykierunek15','INŻYNIERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACEUTYCZNY3','fajnykierunek16','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACEUTYCZNY4','fajnykierunek17','INŻYNIERSKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACUTYCZNY1','fajnykierunek18','LICENCJACKIE');
INSERT INTO Studia (wydzial,kierunek,rodzaj) VALUES ('FARMACEUTYCZNY2','fajnykierunek19','MAGISTERSKIE');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE1');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE2');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE3');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE4');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE5');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE6');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE7');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE8');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE9');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE10');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE11');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE12');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE13');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE14');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE15');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE16');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE17');
INSERT INTO Slowa VALUES ('SLOWOKLUCZOWE18');



(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.616

In [26]:
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('58321',11);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583212',12);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583213',13);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583214',14);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583215',15);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583216',16);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583217',17);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583218',18);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583219',19);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('5832110',20);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('5832111',17);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('5832112',22);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('5832113',18);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583214',25);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583215',25);
INSERT INTO StudenciStudia (id_student,id_studia) VALUES ('583216',26);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.139

In [27]:
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik1','NazwiskoPracownik1','MGR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik2','NazwiskoPracownik2','LIC');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik3','NazwiskoPracownik3','DR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik4','NazwiskoPracownik4','PROF');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik5','NazwiskoPracownik5','MGR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik6','NazwiskoPracownik6','LIC');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik7','NazwiskoPracownik7','PROF');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik18','NazwiskoPracownik8','DR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik19','NazwiskoPracownik9','DR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik20','NazwiskoPracownik10','DR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik21','NazwiskoPracownik11','MGR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik22','NazwiskoPracownik12','MGR');
INSERT INTO Pracownicy (imie,nazwisko,stopien) VALUES ('ImiePracownik23','NazwiskoPracownik13','MGR');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.102

In [28]:
--procedura wyszukująca prace po tematyce; 
CREATE PROCEDURE sp_SzukajPracePoTematyce @temat varchar(100), @temat2 varchar(100)
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_idpromotor int;
DECLARE @v_idszefobrony int;
DECLARE @v_tytul varchar(100);
DECLARE @v_rodzaj varchar(50);
DECLARE @v_wydzial varchar(100);
DECLARE @v_dataobrony datetime;
DECLARE @V_numerstudenta char(10);
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_autorzy varchar(200);

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
        autorzy varchar(200),
        tytul varchar(100),
        promotor varchar(50),
        dataObrony datetime,
        wydzial varchar(100),
        szefObrony varchar(80),
        rodzaj varchar(50)
);
IF @temat2 = ''
	SET @temat2 = @temat;
DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT PraceSlowa.id_praca FROM PraceSlowa WHERE (PraceSlowa.slowo = @temat OR PraceSlowa.slowo = @temat2);
OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy;
WHILE @@FETCH_STATUS = 0
BEGIN
        SELECT @v_tytul=Prace.tytul,@v_idpromotor=Prace.id_promotor,@v_dataobrony=Prace.dataObrony,@v_wydzial=Prace.wydzial,@v_idszefobrony=Prace.id_szefObrony,@v_rodzaj=Prace.grupaRodzaj FROM Prace WHERE Prace.id_praca = @v_numerpracy;
        SELECT @v_promotor=Concat(Pracownicy.stopien, ' ', Pracownicy.imie, ' ', Pracownicy.nazwisko) FROM Pracownicy WHERE Pracownicy.id_pracownik = @v_idpromotor;
        SELECT @v_szefobrony=Concat(Pracownicy.stopien, ' ', Pracownicy.imie, ' ', Pracownicy.nazwisko) FROM Pracownicy WHERE Pracownicy.id_pracownik = @v_idszefobrony;

        SET @v_autorzy = '';
        DECLARE cur2 CURSOR LOCAL FOR SELECT StudenciPrace.id_student FROM StudenciPrace WHERE StudenciPrace.id_praca = @v_numerpracy;
        OPEN cur2;
        FETCH NEXT FROM cur2 INTO @v_numerstudenta;
        WHILE @@FETCH_STATUS = 0
        BEGIN
                SELECT @v_autorzy = @v_autorzy + Studenci.imie + ' ' + Studenci.nazwisko + '; ' FROM Studenci WHERE Studenci.id_student = @v_numerstudenta;
                FETCH NEXT FROM cur2 INTO @v_numerstudenta;
        END;
        CLOSE cur2;
        DEALLOCATE cur2;

        INSERT INTO #ListaPrac VALUES (
                @v_autorzy,
                @v_tytul,
                @v_promotor,
                @v_dataobrony,
                @v_wydzial,
                @v_szefobrony,
                @v_rodzaj
        );

        SET @v_autorzy = '';
        FETCH NEXT FROM cur1 INTO @v_numerpracy;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.048

In [29]:
--procedura szukająca prac po promotorze
CREATE PROCEDURE sp_SzukajPracePoPromotorze @p_imie varchar(20), @p_nazwisko varchar(20), @p_idpromotor int
AS
BEGIN
DECLARE @v_numerpracy int; 
DECLARE @v_idpromotor int;
DECLARE @v_idszefobrony int;
DECLARE @v_tytul varchar(100);
DECLARE @v_rodzaj varchar(50);
DECLARE @v_wydzial varchar(100);
DECLARE @v_dataobrony datetime;
DECLARE @V_numerstudenta char(10);
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_autorzy varchar(200);
DECLARE @v_recenzenci varchar(250);
DECLARE @v_recenzent varchar(80);
DECLARE @v_ocenapromotora decimal(2,1);
DECLARE @v_slowa varchar(400);
DECLARE @v_slowo varchar(100);
DECLARE @v_idpraca int;
IF @p_idpromotor = 0
    DECLARE cur_wybierzPromotorow CURSOR LOCAL FOR SELECT Pracownicy.id_pracownik FROM Pracownicy WHERE (Pracownicy.nazwisko = @p_nazwisko AND Pracownicy.imie = @p_imie);
ELSE
    DECLARE cur_wybierzPromotorow CURSOR LOCAL FOR SELECT Pracownicy.id_pracownik FROM Pracownicy WHERE Pracownicy.id_pracownik = @p_idpromotor;
DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
        id_praca int,
        autorzy varchar(200),
        tytul varchar(100),
        promotor varchar(80),
        ocenaPromotora decimal(2,1),
        dataObrony datetime,
        recenzenci_i_ich_oceny varchar(250),
        wydzial varchar(100),
        szefObrony varchar(80),
        rodzaj varchar(50),
        slowaKluczowe varchar(400)
);
OPEN cur_wybierzPromotorow;
FETCH NEXT FROM cur_wybierzPromotorow INTO @v_idpromotor;
WHILE @@FETCH_STATUS = 0
BEGIN
    SELECT @v_promotor = Concat(Pracownicy.stopien, ' ', Pracownicy.imie, ' ', Pracownicy.nazwisko) FROM Pracownicy WHERE Pracownicy.id_pracownik = @v_idpromotor;
	DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE Prace.id_promotor = @v_idpromotor;
    OPEN cur1;
    FETCH NEXT FROM cur1 INTO @v_numerpracy;
	WHILE @@FETCH_STATUS = 0
    BEGIN
        SELECT @v_tytul=Prace.tytul,@v_dataobrony=Prace.dataObrony,@v_wydzial=Prace.wydzial,@v_idszefobrony=Prace.id_szefObrony,@v_rodzaj=Prace.grupaRodzaj FROM Prace WHERE Prace.id_praca = @v_numerpracy;
        SELECT @v_szefobrony = Concat(Pracownicy.stopien, ' ', Pracownicy.imie, ' ', Pracownicy.nazwisko) FROM Pracownicy WHERE Pracownicy.id_pracownik = @v_idszefobrony;
        SET @v_autorzy = '';
        DECLARE cur2 CURSOR LOCAL FOR SELECT StudenciPrace.id_student FROM StudenciPrace WHERE StudenciPrace.id_praca = @v_numerpracy;
        OPEN cur2;
        FETCH NEXT FROM cur2 INTO @v_numerstudenta;
        WHILE @@FETCH_STATUS = 0
        BEGIN
            SELECT @v_autorzy=Concat(@v_autorzy, Studenci.imie, ' ', Studenci.nazwisko, '; ') FROM Studenci WHERE Studenci.id_student = @v_numerstudenta;
            FETCH NEXT FROM cur2 INTO @v_numerstudenta;
        END;
        CLOSE cur2;
        DEALLOCATE cur2;
        SET @v_recenzenci = '';
	    DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
        OPEN cur3;
	    FETCH NEXT FROM cur3 INTO @v_recenzent;
        WHILE @@FETCH_STATUS = 0
        BEGIN
		    SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
            FETCH NEXT FROM cur3 INTO @v_recenzent;
	    END;
	    CLOSE cur3;
        DEALLOCATE cur3;	
        SELECT @v_ocenapromotora=Prace.ocenaPromotora, @v_idpraca=Prace.id_praca FROM Prace WHERE id_praca = @v_numerpracy;
        
    	SET @v_slowa = '';
        DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE PraceSlowa.id_praca = @v_numerpracy;
	    OPEN cur4;
        FETCH NEXT FROM cur4 INTO @v_slowo;
	    WHILE @@FETCH_STATUS = 0
        BEGIN
    		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
            FETCH NEXT FROM cur4 INTO @v_slowo;
	    END;
	    CLOSE cur4;
        DEALLOCATE cur4;

		INSERT INTO #ListaPrac VALUES (
            @v_idpraca,
			@v_autorzy,
	        @v_tytul,
	        @v_promotor,
            @v_ocenapromotora,
	        @v_dataobrony,
            @v_recenzenci,
	        @v_wydzial,
	        @v_szefobrony,
	        @v_rodzaj,
            @v_slowa
	     );
	     SET @v_autorzy = '';
         FETCH NEXT FROM cur1 INTO @v_numerpracy;
	END;
	CLOSE cur1;
    DEALLOCATE cur1;
    FETCH NEXT FROM cur_wybierzPromotorow INTO @v_idpromotor;
END;
CLOSE cur_wybierzPromotorow;
DEALLOCATE cur_wybierzPromotorow;
SELECT autorzy,tytul,promotor,ocenaPromotora,dataObrony,recenzenci_i_ich_oceny,wydzial,szefObrony,rodzaj,slowaKluczowe FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.091

In [30]:
--procedura szukająca prac danego studenta
CREATE PROCEDURE sp_SzukajPracPoStudencie @p_idstudent char(10), @p_nazwisko varchar(20), @p_imie varchar(20)
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_liczbaRobotnikow int;
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_recenzent varchar(80);
DECLARE @v_wspolautor varchar(80);
DECLARE @v_slowo varchar(80);
DECLARE @v_recenzenci varchar(300);
DECLARE @v_wspolautorzy varchar(300);
DECLARE @v_slowa varchar(300);
DECLARE @v_rodzaj varchar (50);
DECLARE @v_ocena decimal(2,1);
DECLARE @v_ocenaPromotora decimal(2,1);
DECLARE @v_dataObrony datetime;
DECLARE @v_tytul varchar(100);
DECLARE @v_wydzial varchar(100);
DECLARE @wiadomosc varchar(100);

IF @p_idstudent != ''
BEGIN
	DECLARE cur1 CURSOR LOCAL FOR SELECT StudenciPrace.id_praca, StudenciPrace.ocena FROM StudenciPrace WHERE StudenciPrace.id_student = @p_idstudent;
	SELECT @wiadomosc = Concat('Wyświetla prace studenta ',Studenci.imie,' ',Studenci.nazwisko) FROM Studenci WHERE id_student = @p_idstudent;
END
ELSE
BEGIN
	DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT StudenciPrace.id_praca, StudenciPrace.ocena FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE (Studenci.nazwisko = @p_nazwisko AND Studenci.imie = @p_imie);
	SELECT @wiadomosc = Concat('Wyświetla prace studentów o imionach i nazwiskach: ', @p_imie, ' ', @p_nazwisko);
END;

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
	tytul varchar (100),
	wspolautorzy varchar(300),
	promotor varchar(80),
	recenzenci_i_ich_oceny varchar(300),
	ocena_promotora decimal(2,1),
	ocena_koncowa decimal(2,1),
	data_obrony datetime,
	wydzial varchar(100),
	rodzaj_pracy varchar(50),
	slowa_kluczowe varchar(300)
);

PRINT @wiadomosc;

OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy, @v_ocena;
WHILE @@FETCH_STATUS = 0
BEGIN
	SELECT @v_promotor = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_promotor = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SELECT @v_szefobrony = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_szefObrony = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SET @v_recenzenci = '';
	DECLARE cur2 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
    OPEN cur2;
	FETCH NEXT FROM cur2 INTO @v_recenzent;
    WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
        FETCH NEXT FROM cur2 INTO @v_recenzent;
	END;
	CLOSE cur2;
    DEALLOCATE cur2;	
	SELECT @v_liczbaRobotnikow = Prace.liczbaRobotnikow FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	IF @v_liczbaRobotnikow > 1
    BEGIN
		SET @v_wspolautorzy = '';
        DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Studenci.imie,' ',Studenci.nazwisko) FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE (StudenciPrace.id_praca = @v_numerpracy AND StudenciPrace.id_student != @p_idstudent);
		OPEN cur3;
        FETCH NEXT FROM cur3 INTO @v_wspolautor;
		WHILE @@FETCH_STATUS = 0
        BEGIN
			SET @v_wspolautorzy = Concat(@v_wspolautorzy,@v_wspolautor,';');
            FETCH NEXT FROM cur3 INTO @v_wspolautor;
		END;
		CLOSE cur3;
        DEALLOCATE cur3;
    END
	ELSE
		SET @v_wspolautorzy = 'BRAK';
	SET @v_slowa = '';
    DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE PraceSlowa.id_praca = @v_numerpracy;
	OPEN cur4;
    FETCH NEXT FROM cur4 INTO @v_slowo;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
        FETCH NEXT FROM cur4 INTO @v_slowo;
	END;
	CLOSE cur4;
    DEALLOCATE cur4;	
    SELECT @v_rodzaj=Prace.grupaRodzaj, @v_ocenaPromotora=Prace.ocenaPromotora, @v_dataObrony=Prace.dataObrony, @v_tytul=Prace.tytul, @v_wydzial=Prace.wydzial FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	INSERT INTO #ListaPrac(tytul,wspolautorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,ocena_koncowa,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe) VALUES (
		@v_tytul,
		@v_wspolautorzy,
		@v_promotor,
		@v_recenzenci,
		@v_ocenaPromotora,
		@v_ocena,
		@v_dataObrony,
		@v_wydzial,
		@v_rodzaj,
		@v_slowa
	);	
	FETCH NEXT FROM cur1 INTO @v_numerpracy, @v_ocena;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.060

In [31]:
--procedura szukająca prac po danym kierunku lub roku obrony
CREATE PROCEDURE sp_SzukajPracePoKierunkuLubRoku @p_kierunek varchar(100), @p_rok int
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_recenzent varchar(80);
DECLARE @v_wspolautor varchar(80);
DECLARE @v_slowo varchar(80);
DECLARE @v_recenzenci varchar (300);
DECLARE @v_wspolautorzy varchar(300);
DECLARE @v_slowa varchar(300);
DECLARE @v_rodzaj varchar (50);
DECLARE @v_ocena decimal(2,1);
DECLARE @v_ocenaPromotora decimal(2,1);
DECLARE @v_dataObrony datetime;
DECLARE @v_tytul varchar(100);
DECLARE @v_wydzial varchar(100);
DECLARE @wiadomosc varchar(100);

DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT StudenciPrace.id_praca FROM StudenciPrace
	INNER JOIN StudenciStudia ON StudenciPrace.id_student_studia = StudenciStudia.id_student_studia
	INNER JOIN Studia ON StudenciStudia.id_studia = Studia.id_studia
	WHERE Studia.kierunek = @p_kierunek
	UNION
	SELECT Prace.id_praca FROM Prace WHERE YEAR(Prace.dataObrony) = @p_rok;

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
	tytul varchar (100),
	autorzy varchar(300),
	promotor varchar(80),
	recenzenci_i_ich_oceny varchar(300),
	ocena_promotora decimal(2,1),
	data_obrony datetime,
	wydzial varchar(100),
	rodzaj_pracy varchar(50),
	slowa_kluczowe varchar(300)
);

SET @wiadomosc = Concat('Prace obronione na kierunku: ',@p_kierunek,' lub w roku ',@p_rok);
PRINT @wiadomosc;

OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy;
WHILE @@FETCH_STATUS = 0
BEGIN
	SELECT @v_promotor = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_promotor = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SELECT @v_szefobrony = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_szefObrony = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SET @v_recenzenci = '';
	DECLARE cur2 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
    OPEN cur2;
	FETCH NEXT FROM cur2 INTO @v_recenzent;
    WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
        FETCH NEXT FROM cur2 INTO @v_recenzent;
	END;
	CLOSE cur2;
    DEALLOCATE cur2;
	SET @v_wspolautorzy = '';
	DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Studenci.imie,' ',Studenci.nazwisko) FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE StudenciPrace.id_praca = @v_numerpracy;
    OPEN cur3;
    FETCH NEXT FROM cur3 INTO @v_wspolautor;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_wspolautorzy = Concat(@v_wspolautorzy,@v_wspolautor,';');
        FETCH NEXT FROM cur3 INTO @v_wspolautor;
	END;
	CLOSE cur3;
    DEALLOCATE cur3;	
	SET @v_slowa = '';
	DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE id_praca = @v_numerpracy;
    OPEN cur4;
    FETCH NEXT FROM cur4 INTO @v_slowo;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
        FETCH NEXT FROM cur4 INTO @v_slowo;
	END;
	CLOSE cur4;
    DEALLOCATE cur4;
	
	SELECT @v_rodzaj=Prace.grupaRodzaj, @v_ocenaPromotora=Prace.ocenaPromotora, @V_dataObrony=Prace.dataObrony, @v_tytul=Prace.tytul, @v_wydzial=Prace.wydzial FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	INSERT INTO #ListaPrac(tytul,autorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe) VALUES (
		@v_tytul,
		@v_wspolautorzy,
		@v_promotor,
		@v_recenzenci,
		@v_ocenaPromotora,
		@v_dataObrony,
		@v_wydzial,
		@v_rodzaj,
		@v_slowa
	);	
	FETCH NEXT FROM cur1 INTO @v_numerpracy;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.050

In [32]:
--procedure szukająca prac recenzenzowanych
CREATE PROCEDURE sp_SzukajPraceRecenzowane @p_idpracownik int, @p_imiepracownik varchar(20), @p_nazwiskopracownik varchar(20)
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_liczbaRobotnikow int;
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_recenzent varchar(80);
DECLARE @v_wspolautor varchar(80);
DECLARE @v_slowo varchar(80);
DECLARE @v_recenzenci varchar(300);
DECLARE @v_wspolautorzy varchar(400);
DECLARE @v_slowa varchar(300);
DECLARE @v_rodzaj varchar (50);
DECLARE @v_ocena decimal(2,1);
DECLARE @v_ocenaPromotora decimal(2,1);
DECLARE @v_dataObrony datetime;
DECLARE @v_tytul varchar(100);
DECLARE @v_wydzial varchar(100);
DECLARE @wiadomosc varchar(100);

IF ((@p_idpracownik != 0) AND ((@p_imiepracownik != '' AND @p_nazwiskopracownik != '')))
    DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT PraceRecenzenci.id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.nazwisko = @p_nazwiskopracownik AND Pracownicy.imie = @p_imiepracownik) OR (Pracownicy.id_pracownik = @p_idpracownik));
ELSE IF ((@p_idpracownik != 0) AND ((@p_imiepracownik = '' OR @p_imiepracownik = '')))
BEGIN
    IF (@p_nazwiskopracownik = '' AND @p_imiepracownik != '')
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.imie = @p_imiepracownik) OR (Pracownicy.id_pracownik = @p_idpracownik));    
    ELSE IF (@p_imiepracownik = '' AND @p_nazwiskopracownik != '')
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.nazwisko = @p_nazwiskopracownik) OR (Pracownicy.id_pracownik = @p_idpracownik));
    ELSE
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE (Pracownicy.id_pracownik = @p_idpracownik);    
END;
ELSE IF ((@p_idpracownik = 0) AND (@p_imiepracownik != '' AND @p_nazwiskopracownik != ''))
    DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.imie = @p_imiepracownik AND Pracownicy.nazwisko = @p_nazwiskopracownik));
ELSE IF ((@p_idpracownik = 0) AND ((@p_imiepracownik = '' OR @p_nazwiskopracownik = '')))
BEGIN
    IF (@p_nazwiskopracownik = '' AND @p_imiepracownik != '')
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.imie = @p_imiepracownik));    
    ELSE IF (@p_imiepracownik = '' AND @p_nazwiskopracownik != '')
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE ((Pracownicy.nazwisko = @p_nazwiskopracownik));
    ELSE
        DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci;    
END;
ELSE
    DECLARE cur1 CURSOR LOCAL FOR SELECT DISTINCT id_praca FROM PraceRecenzenci; 

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
	tytul varchar (100),
	autorzy_i_ich_oceny varchar(300),
	promotor varchar(80),
	recenzenci_i_ich_oceny varchar(300),
	ocena_promotora decimal(2,1),
	data_obrony datetime,
	wydzial varchar(100),
	rodzaj_pracy varchar(50),
	slowa_kluczowe varchar(300)
);

SELECT @wiadomosc = 'Wyświetla prace recenzowane przez określonych pracowników uczelni';
PRINT @wiadomosc;

OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy;
WHILE @@FETCH_STATUS = 0
BEGIN
	SELECT @v_promotor = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_promotor = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SELECT @v_szefobrony = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_szefObrony = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SET @v_recenzenci = '';
	DECLARE cur2 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
    OPEN cur2;
	FETCH NEXT FROM cur2 INTO @v_recenzent;
    WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
        FETCH NEXT FROM cur2 INTO @v_recenzent;
	END;
	CLOSE cur2;
    DEALLOCATE cur2;
	SET @v_wspolautorzy = '';
    DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Studenci.imie,' ',Studenci.nazwisko,' ',StudenciPrace.ocena) FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE (StudenciPrace.id_praca = @v_numerpracy);
	OPEN cur3;
    FETCH NEXT FROM cur3 INTO @v_wspolautor;
	WHILE @@FETCH_STATUS = 0
    BEGIN
	    SET @v_wspolautorzy = Concat(@v_wspolautorzy,@v_wspolautor,';');
        FETCH NEXT FROM cur3 INTO @v_wspolautor;
	END;
	CLOSE cur3;
    DEALLOCATE cur3;
	SET @v_slowa = '';
    DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE PraceSlowa.id_praca = @v_numerpracy;
	OPEN cur4;
    FETCH NEXT FROM cur4 INTO @v_slowo;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
        FETCH NEXT FROM cur4 INTO @v_slowo;
	END;
	CLOSE cur4;
    DEALLOCATE cur4;	
    SELECT @v_rodzaj=Prace.grupaRodzaj, @v_ocenaPromotora=Prace.ocenaPromotora, @v_dataObrony=Prace.dataObrony, @v_tytul=Prace.tytul, @v_wydzial=Prace.wydzial FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	INSERT INTO #ListaPrac(tytul,autorzy_i_ich_oceny,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe) VALUES (
		@v_tytul,
		@v_wspolautorzy,
		@v_promotor,
		@v_recenzenci,
		@v_ocenaPromotora,
		@v_dataObrony,
		@v_wydzial,
		@v_rodzaj,
		@v_slowa
	);	
	FETCH NEXT FROM cur1 INTO @v_numerpracy;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.094

In [33]:
--procedura szukająca prac obronionych po datach
CREATE PROCEDURE sp_SzukajPracePoDacie @p_rok int, @p_miesiac int, @p_dzien int
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_liczbaRobotnikow int;
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_recenzent varchar(80);
DECLARE @v_wspolautor varchar(80);
DECLARE @v_slowo varchar(80);
DECLARE @v_recenzenci varchar(300);
DECLARE @v_wspolautorzy varchar(400);
DECLARE @v_slowa varchar(300);
DECLARE @v_rodzaj varchar (50);
DECLARE @v_ocena decimal(2,1);
DECLARE @v_ocenaPromotora decimal(2,1);
DECLARE @v_dataObrony datetime;
DECLARE @v_tytul varchar(100);
DECLARE @v_wydzial varchar(100);
DECLARE @wiadomosc varchar(100);

IF ((@p_rok != 0) AND ((@p_miesiac != 0 AND @p_dzien != 0)))
    DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE ((YEAR(Prace.dataObrony) = @p_rok AND MONTH(Prace.dataObrony) = @p_miesiac) AND (DAY(Prace.dataObrony) = @p_dzien));
ELSE IF ((@p_rok != 0) AND ((@p_miesiac = 0 OR @p_dzien = 0)))
BEGIN
    IF (@p_miesiac = 0 AND @p_dzien != 0)
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE ((YEAR(Prace.dataObrony) = @p_rok AND DAY(Prace.dataObrony) = @p_dzien));
    ELSE IF (@p_dzien = 0 AND @p_miesiac != 0)
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE ((YEAR(Prace.dataObrony) = @p_rok AND MONTH(Prace.dataObrony) = @p_miesiac));
    ELSE
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE ((YEAR(Prace.dataObrony) = @p_rok));
END;
ELSE IF ((@p_rok = 0) AND (@p_miesiac != 0 AND @p_dzien != 0))
    DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE (((MONTH(Prace.dataObrony) = @p_miesiac) AND (DAY(Prace.dataObrony) = @p_dzien)));
ELSE IF ((@p_rok = 0) AND ((@p_miesiac = 0 OR @p_dzien = 0)))
BEGIN
    IF (@p_miesiac = 0 AND @p_dzien != 0)
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE (DAY(Prace.dataObrony) = @p_dzien);
    ELSE IF (@p_dzien = 0 AND @p_miesiac != 0)
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace WHERE (MONTH(Prace.dataObrony) = @p_miesiac);
    ELSE
        DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace;    
END;
ELSE
    DECLARE cur1 CURSOR LOCAL FOR SELECT Prace.id_praca FROM Prace;  

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
	tytul varchar (100),
	autorzy_i_ich_oceny varchar(300),
	promotor varchar(80),
	recenzenci_i_ich_oceny varchar(300),
	ocena_promotora decimal(2,1),
	data_obrony datetime,
	wydzial varchar(100),
	rodzaj_pracy varchar(50),
	slowa_kluczowe varchar(300)
);

SELECT @wiadomosc = 'Wyświetla prace obronione w konkretnych datach';
PRINT @wiadomosc;

OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy;
WHILE @@FETCH_STATUS = 0
BEGIN
	SELECT @v_promotor = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_promotor = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SELECT @v_szefobrony = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_szefObrony = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SET @v_recenzenci = '';
	DECLARE cur2 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
    OPEN cur2;
	FETCH NEXT FROM cur2 INTO @v_recenzent;
    WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
        FETCH NEXT FROM cur2 INTO @v_recenzent;
	END;
	CLOSE cur2;
    DEALLOCATE cur2;
	SET @v_wspolautorzy = '';
    DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Studenci.imie,' ',Studenci.nazwisko,' ',StudenciPrace.ocena) FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE (StudenciPrace.id_praca = @v_numerpracy);
	OPEN cur3;
    FETCH NEXT FROM cur3 INTO @v_wspolautor;
	WHILE @@FETCH_STATUS = 0
    BEGIN
	    SET @v_wspolautorzy = Concat(@v_wspolautorzy,@v_wspolautor,';');
        FETCH NEXT FROM cur3 INTO @v_wspolautor;
	END;
	CLOSE cur3;
    DEALLOCATE cur3;
	SET @v_slowa = '';
    DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE PraceSlowa.id_praca = @v_numerpracy;
	OPEN cur4;
    FETCH NEXT FROM cur4 INTO @v_slowo;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
        FETCH NEXT FROM cur4 INTO @v_slowo;
	END;
	CLOSE cur4;
    DEALLOCATE cur4;	
    SELECT @v_rodzaj=Prace.grupaRodzaj, @v_ocenaPromotora=Prace.ocenaPromotora, @v_dataObrony=Prace.dataObrony, @v_tytul=Prace.tytul, @v_wydzial=Prace.wydzial FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	INSERT INTO #ListaPrac(tytul,autorzy_i_ich_oceny,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe) VALUES (
		@v_tytul,
		@v_wspolautorzy,
		@v_promotor,
		@v_recenzenci,
		@v_ocenaPromotora,
		@v_dataObrony,
		@v_wydzial,
		@v_rodzaj,
		@v_slowa
	);	
	FETCH NEXT FROM cur1 INTO @v_numerpracy;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;


Commands completed successfully.

Total execution time: 00:00:00.058

In [34]:
--procedura szukająca prac po rodzaju studiów
CREATE PROCEDURE sp_SzukajPracePoRodzaju @p_rodzaj varchar(50)
AS
BEGIN
DECLARE @v_numerpracy int;
DECLARE @v_promotor varchar(80);
DECLARE @v_szefobrony varchar(80);
DECLARE @v_recenzent varchar(80);
DECLARE @v_wspolautor varchar(80);
DECLARE @v_slowo varchar(80);
DECLARE @v_recenzenci varchar (300);
DECLARE @v_wspolautorzy varchar(300);
DECLARE @v_slowa varchar(300);
DECLARE @v_rodzaj varchar (50);
DECLARE @v_ocena decimal(2,1);
DECLARE @v_ocenaPromotora decimal(2,1);
DECLARE @v_dataObrony datetime;
DECLARE @v_tytul varchar(100);
DECLARE @v_wydzial varchar(100);
DECLARE @wiadomosc varchar(100);

DECLARE cur1 CURSOR LOCAL FOR SELECT id_praca FROM Prace WHERE grupaRodzaj = @p_rodzaj;

DROP TABLE IF EXISTS #ListaPrac;
CREATE TABLE #ListaPrac (
	tytul varchar (100),
	autorzy varchar(300),
	promotor varchar(80),
	recenzenci_i_ich_oceny varchar(300),
	ocena_promotora decimal(2,1),
	data_obrony datetime,
	wydzial varchar(100),
	rodzaj_pracy varchar(50),
	slowa_kluczowe varchar(300)
);

SET @wiadomosc = 'Wyświetla prace określonego rodzaju';
PRINT @wiadomosc;

OPEN cur1;
FETCH NEXT FROM cur1 INTO @v_numerpracy;
WHILE @@FETCH_STATUS = 0
BEGIN
	SELECT @v_promotor = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_promotor = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SELECT @v_szefobrony = Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko) FROM Pracownicy INNER JOIN Prace ON Prace.id_szefObrony = Pracownicy.id_pracownik WHERE Prace.id_praca = @v_numerpracy;
	SET @v_recenzenci = '';
	DECLARE cur2 CURSOR LOCAL FOR SELECT Concat(Pracownicy.stopien,' ',Pracownicy.imie,' ',Pracownicy.nazwisko,', ocena: ',PraceRecenzenci.ocena) FROM PraceRecenzenci INNER JOIN Pracownicy ON PraceRecenzenci.id_recenzent = Pracownicy.id_pracownik WHERE PraceRecenzenci.id_praca = @v_numerpracy;
    OPEN cur2;
	FETCH NEXT FROM cur2 INTO @v_recenzent;
    WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_recenzenci = Concat(@v_recenzenci, @v_recenzent,';');
        FETCH NEXT FROM cur2 INTO @v_recenzent;
	END;
	CLOSE cur2;
    DEALLOCATE cur2;
	SET @v_wspolautorzy = '';
	DECLARE cur3 CURSOR LOCAL FOR SELECT Concat(Studenci.imie,' ',Studenci.nazwisko) FROM StudenciPrace INNER JOIN Studenci ON StudenciPrace.id_student = Studenci.id_student WHERE StudenciPrace.id_praca = @v_numerpracy;
    OPEN cur3;
    FETCH NEXT FROM cur3 INTO @v_wspolautor;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_wspolautorzy = Concat(@v_wspolautorzy,@v_wspolautor,';');
        FETCH NEXT FROM cur3 INTO @v_wspolautor;
	END;
	CLOSE cur3;
    DEALLOCATE cur3;	
	SET @v_slowa = '';
	DECLARE cur4 CURSOR LOCAL FOR SELECT PraceSlowa.slowo FROM PraceSlowa WHERE id_praca = @v_numerpracy;
    OPEN cur4;
    FETCH NEXT FROM cur4 INTO @v_slowo;
	WHILE @@FETCH_STATUS = 0
    BEGIN
		SET @v_slowa = Concat(@v_slowa,@v_slowo,'; ');
        FETCH NEXT FROM cur4 INTO @v_slowo;
	END;
	CLOSE cur4;
    DEALLOCATE cur4;
	
	SELECT @v_rodzaj=Prace.grupaRodzaj, @v_ocenaPromotora=Prace.ocenaPromotora, @V_dataObrony=Prace.dataObrony, @v_tytul=Prace.tytul, @v_wydzial=Prace.wydzial FROM Prace WHERE Prace.id_praca = @v_numerpracy;
	INSERT INTO #ListaPrac(tytul,autorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe) VALUES (
		@v_tytul,
		@v_wspolautorzy,
		@v_promotor,
		@v_recenzenci,
		@v_ocenaPromotora,
		@v_dataObrony,
		@v_wydzial,
		@v_rodzaj,
		@v_slowa
	);	
	FETCH NEXT FROM cur1 INTO @v_numerpracy;
END;
CLOSE cur1;
DEALLOCATE cur1;
SELECT * FROM #ListaPrac;
DROP TABLE #ListaPrac;
END;

Commands completed successfully.

Total execution time: 00:00:00.052

In [35]:
--INDEKSY:
--Prawie wszystkie pożądane indeksy są dodane automatycznie przez DBMS jako clustered na kolumnach kluczy pierwotnych/
--oraz jako non-clustered na kolumnach z unique constraint./
--Jedyne potrzebne dodatkowe indeksy to indeksy typu non-clustered na polach po których odbywa się szukanie w procedurach i łączenie tabel
--Dodawanie innych indeksów jedynie spowolni dodawanie, usuwanie i update danych.
--celem przyspieszenia wyszukiwania
CREATE NONCLUSTERED INDEX ind_Prace_idpromotor ON Prace(id_promotor);
CREATE NONCLUSTERED INDEX ind_Prace_dataobrony ON Prace(dataObrony);
CREATE NONCLUSTERED INDEX ind_Studia_kierunek ON Studia(kierunek);
CREATE NONCLUSTERED INDEX ind_PraceRecenzenci_id_recenzent ON PraceRecenzenci(id_recenzent);
CREATE NONCLUSTERED INDEX ind_PraceSlowa_slowo ON PraceSlowa(slowo);
CREATE NONCLUSTERED INDEX ind_Pracownicy_nazwisko ON Pracownicy(nazwisko);
CREATE NONCLUSTERED INDEX ind_Studenci_nazwisko ON Studenci(nazwisko);
--celem przyspieszenia łączenia tabel w zapytaniach
CREATE NONCLUSTERED INDEX ind_StudenciStudia_idstudent ON StudenciStudia(id_student);
CREATE NONCLUSTERED INDEX ind_StudenciStudia_idstudia ON StudenciStudia(id_studia);
CREATE NONCLUSTERED INDEX ind_StudenciPrace_idpraca ON StudenciPrace(id_praca);
CREATE NONCLUSTERED INDEX ind_StudenciPrace_idstudentstudia ON StudenciPrace(id_student_studia);

Commands completed successfully.

Total execution time: 00:00:00.135

In [36]:
--demonstracja szukania po rodzaju (nr 7 z harmonogramu przedmiotu, obecna w założeniach projektu, nieobecna w poleceniu E3)
EXEC sp_SzukajPracePoRodzaju 'LICENCJACKIE';

Wyswietla prace okreslonego rodzaju

(1 row affected)

(1 row affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:03.813

tytul,autorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Opieka nad pacjentami z choroba Crohna,Eleonora Elektryfikujaca;Franciszek Ferdynancki;Grzegorz Gorycki;,DR Zygfryd Zebrzanski,"PROF Rakoczy Rondecki, ocena: 4.0;DR HAB Paulina Penderecka, ocena: 3.0;","3,0",2021-03-12 00:00:00.000,NAUK O ZDROWIU,LICENCJACKIE,NITROZAMINY; PRACA DYPLOMOWA;
Sily odsrodkowe w autobusach,Czeslawa Cymbalinska;,DR HAB Paulina Penderecka,"PROF Rakoczy Rondecki, ocena: 5.0;MGR Tadeusz Tokarski, ocena: 4.5;INZ Ola Oskarska, ocena: 3.5;","5,0",2021-06-21 00:00:00.000,FIZYKI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; PRACA DYPLOMOWA;
Inflacja w krajach Europy Srodkowo-Wschodniej,Dionizy Doglebny;,DR INZ Marek Moczydlowski,"PROF Rakoczy Rondecki, ocena: 3.5;","4,5",2018-12-21 00:00:00.000,EKONOMII I ADMINISTRACJI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; POTENCJAL BLONOWY; PRACA DYPLOMOWA;


In [37]:
--demonstracja szukania po dacie (nieobecna w harmonogramie przedmiotu, obecna w założeniach projektu, nieobecna w poleceniu E3)
EXEC sp_SzukajPracePoDacie 0,6,0;

Wyswietla prace obronione w konkretnych datach

(1 row affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.320

tytul,autorzy_i_ich_oceny,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Kwantowa teoria pola w czarnych dziurach,Henryk Heniawski 3.5;Franciszek Frankowski 3.5;,PROF Rakoczy Rondecki,"DR HAB Paulina Penderecka, ocena: 5.0;","4,0",2019-06-11 00:00:00.000,FIZYKI,DOKTORANCKIE,NEUROLEPTYKI; PRACA DYPLOMOWA;
Sily odsrodkowe w autobusach,Czeslawa Cymbalinska 3.5;,DR HAB Paulina Penderecka,"PROF Rakoczy Rondecki, ocena: 5.0;MGR Tadeusz Tokarski, ocena: 4.5;INZ Ola Oskarska, ocena: 3.5;","5,0",2021-06-21 00:00:00.000,FIZYKI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; PRACA DYPLOMOWA;


In [38]:
--demonstracja szukania po recenzencie (nieobecna w harmonogramie przedmiotu, obecna w założeniach projektu, nieobecna w poleceniu E3)
EXEC sp_SzukajPraceRecenzowane 0,'Tadeusz','';

Wyswietla prace recenzowane przez okreslonych pracowników uczelni

(1 row affected)

(1 row affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.411

tytul,autorzy_i_ich_oceny,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Sily odsrodkowe w autobusach,Czeslawa Cymbalinska 3.5;,DR HAB Paulina Penderecka,"PROF Rakoczy Rondecki, ocena: 5.0;MGR Tadeusz Tokarski, ocena: 4.5;INZ Ola Oskarska, ocena: 3.5;","5,0",2021-06-21 00:00:00.000,FIZYKI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; PRACA DYPLOMOWA;
Pomiary napiecia w procesorach x86,Bartosz Benedyktynski 3.5;,PROF Rakoczy Rondecki,"MGR Tadeusz Tokarski, ocena: 4.5;","3,5",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;
Zastosowania kryptografii na stronach rzadowych,Anna Andrzejewska 5.0;,DR INZ Marek Moczydlowski,"MGR Tadeusz Tokarski, ocena: 3.5;","3,0",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [39]:
--demonstracja szukania po kierunku i roku (nieobecna w harmonogramie przedmiotu, nieobecna w założeniach projektu, obecna w poleceniu E3)
EXEC sp_SzukajPracePoKierunkuLubRoku 'CHEMIA ORGANICZNA', 2018

Prace obronione na kierunku: CHEMIA ORGANICZNA lub w roku 2018

(1 row affected)

(1 row affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.077

tytul,autorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Nitrozaminy w komórkach ludzkich,Bartosz Benedyktynski;,PROF Rakoczy Rondecki,"DR Zygfryd Zebrzanski, ocena: 5.0;","3,0",2020-03-12 00:00:00.000,CHEMII,MAGISTERSKIE,KONSTRUKCJE ZELBETOWE; NITROZAMINY; PRACA DYPLOMOWA;
Inflacja w krajach Europy Srodkowo-Wschodniej,Dionizy Doglebny;,DR INZ Marek Moczydlowski,"PROF Rakoczy Rondecki, ocena: 3.5;","4,5",2018-12-21 00:00:00.000,EKONOMII I ADMINISTRACJI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; POTENCJAL BLONOWY; PRACA DYPLOMOWA;
Elektrochemia kardiomiocytów,Ilona Ilanicka;,PROF Marek Moczydlowski,"LIC Sonia Szarecka, ocena: 3.5;DR INZ Marek Moczydlowski, ocena: 4.5;","5,0",2020-02-12 00:00:00.000,CHEMII,MAGISTERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [40]:
--demonstracja szukania po studencie (nieobecna w harmonogramie przedmiotu, nieobecna w założeniach projektu, obecna w poleceniu E3)
EXEC sp_SzukajPracPoStudencie '138', '', '';

Wyswietla prace studenta Bartosz Benedyktynski

(1 row affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.492

tytul,wspolautorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,ocena_koncowa,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Nitrozaminy w komórkach ludzkich,BRAK,PROF Rakoczy Rondecki,"DR Zygfryd Zebrzanski, ocena: 5.0;","3,0","4,0",2020-03-12 00:00:00.000,CHEMII,MAGISTERSKIE,KONSTRUKCJE ZELBETOWE; NITROZAMINY; PRACA DYPLOMOWA;
Pomiary napiecia w procesorach x86,BRAK,PROF Rakoczy Rondecki,"MGR Tadeusz Tokarski, ocena: 4.5;","3,5","3,5",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [41]:
--demonstracja szukania po promotorze (nr 13 z harmonogramu przedmiotu, obecna w założeniach projektu, obecna w poleceniu E3)
EXEC sp_SzukajPracePoPromotorze 'Marek', 'Moczydłowski', 0;

(1 row affected)

(1 row affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.354

autorzy,tytul,promotor,ocenaPromotora,dataObrony,recenzenci_i_ich_oceny,wydzial,szefObrony,rodzaj,slowaKluczowe
Dionizy Doglebny;,Inflacja w krajach Europy Srodkowo-Wschodniej,DR INZ Marek Moczydlowski,"4,5",2018-12-21 00:00:00.000,"PROF Rakoczy Rondecki, ocena: 3.5;",EKONOMII I ADMINISTRACJI,DR HAB Paulina Penderecka,LICENCJACKIE,NOSOROZCE NA SAWANNIE; POTENCJAL BLONOWY; PRACA DYPLOMOWA;
Anna Andrzejewska;,Zastosowania kryptografii na stronach rzadowych,DR INZ Marek Moczydlowski,"3,0",2020-01-10 00:00:00.000,"MGR Tadeusz Tokarski, ocena: 3.5;",ELEKTRYCZNY,PROF Rakoczy Rondecki,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;
Ilona Ilanicka;,Elektrochemia kardiomiocytów,PROF Marek Moczydlowski,"5,0",2020-02-12 00:00:00.000,"LIC Sonia Szarecka, ocena: 3.5;DR INZ Marek Moczydlowski, ocena: 4.5;",CHEMII,DR Zygfryd Zebrzanski,MAGISTERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [42]:
--demonstracja szukania po tematyce (nr 8 z harmonogramu przedmiotu, obecna w założeniach projektu, obecna w poleceniu E3)
EXEC sp_SzukajPracePoTematyce 'NITROZAMINY', 'NOSOROŻCE NA SAWANNIE';

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.079

autorzy,tytul,promotor,dataObrony,wydzial,szefObrony,rodzaj
Bartosz Benedyktynski;,Nitrozaminy w komórkach ludzkich,PROF Rakoczy Rondecki,2020-03-12 00:00:00.000,CHEMII,DR INZ Marek Moczydlowski,MAGISTERSKIE
Eleonora Elektryfikujaca; Franciszek Ferdynancki; Grzegorz Gorycki;,Opieka nad pacjentami z choroba Crohna,DR Zygfryd Zebrzanski,2021-03-12 00:00:00.000,NAUK O ZDROWIU,INZ Ola Oskarska,LICENCJACKIE
Czeslawa Cymbalinska;,Sily odsrodkowe w autobusach,DR HAB Paulina Penderecka,2021-06-21 00:00:00.000,FIZYKI,MGR Tadeusz Tokarski,LICENCJACKIE
Dionizy Doglebny;,Inflacja w krajach Europy Srodkowo-Wschodniej,DR INZ Marek Moczydlowski,2018-12-21 00:00:00.000,EKONOMII I ADMINISTRACJI,DR HAB Paulina Penderecka,LICENCJACKIE


In [43]:
--tworzenie widoków celem poprawy bezpieczeństwa w zakresie dostępu do danych
--widok dla recenzenta na prace do recenzji/zrecenzowane
CREATE VIEW vw_Recenzent_RecenzjePrac
AS
SELECT 
    Prace.id_praca, Prace.tytul, Prace.grupaRodzaj, Prace.wydzial, 
    PraceRecenzenci.id_recenzent, PraceRecenzenci.ocena
FROM Prace 
INNER JOIN PraceRecenzenci ON Prace.id_praca = PraceRecenzenci.id_praca;
GO
--widok dla szefa obrony na studentów broniących się/którzy obronili swoją pracę
CREATE VIEW vw_SzefObrony_ObronyPrac
AS
SELECT
    Prace.id_praca, Prace.tytul, Prace.grupaRodzaj, Prace.wydzial, Prace.dataObrony, Prace.ocenaKoncowa,
    StudenciPrace.id_student, StudenciPrace.ocena
FROM Prace
INNER JOIN StudenciPrace ON StudenciPrace.id_praca = Prace.id_praca;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.098

In [44]:
--test powyższych widoków
SELECT * FROM vw_Recenzent_RecenzjePrac;
SELECT * FROM vw_SzefObrony_ObronyPrac;

(12 rows affected)

(11 rows affected)

Total execution time: 00:00:00.052

id_praca,tytul,grupaRodzaj,wydzial,id_recenzent,ocena
1,Nitrozaminy w komórkach ludzkich,MAGISTERSKIE,CHEMII,3,"5,0"
2,Opieka nad pacjentami z choroba Crohna,LICENCJACKIE,NAUK O ZDROWIU,2,"4,0"
2,Opieka nad pacjentami z choroba Crohna,LICENCJACKIE,NAUK O ZDROWIU,4,"3,0"
3,Kwantowa teoria pola w czarnych dziurach,DOKTORANCKIE,FIZYKI,4,"5,0"
4,Sily odsrodkowe w autobusach,LICENCJACKIE,FIZYKI,2,"5,0"
4,Sily odsrodkowe w autobusach,LICENCJACKIE,FIZYKI,5,"4,5"
4,Sily odsrodkowe w autobusach,LICENCJACKIE,FIZYKI,7,"3,5"
5,Inflacja w krajach Europy Srodkowo-Wschodniej,LICENCJACKIE,EKONOMII I ADMINISTRACJI,2,"3,5"
6,Pomiary napiecia w procesorach x86,INZYNIERSKIE,ELEKTRYCZNY,5,"4,5"
7,Zastosowania kryptografii na stronach rzadowych,INZYNIERSKIE,ELEKTRYCZNY,5,"3,5"


id_praca,tytul,grupaRodzaj,wydzial,dataObrony,ocenaKoncowa,id_student,ocena
2,Opieka nad pacjentami z choroba Crohna,LICENCJACKIE,NAUK O ZDROWIU,2021-03-12 00:00:00.000,"3,0",1123,"3,5"
3,Kwantowa teoria pola w czarnych dziurach,DOKTORANCKIE,FIZYKI,2019-06-11 00:00:00.000,"3,0",11234,"3,5"
7,Zastosowania kryptografii na stronach rzadowych,INZYNIERSKIE,ELEKTRYCZNY,2020-01-10 00:00:00.000,"3,0",135,"5,0"
1,Nitrozaminy w komórkach ludzkich,MAGISTERSKIE,CHEMII,2020-03-12 00:00:00.000,"3,0",138,"4,0"
6,Pomiary napiecia w procesorach x86,INZYNIERSKIE,ELEKTRYCZNY,2020-01-10 00:00:00.000,"3,0",138,"3,5"
3,Kwantowa teoria pola w czarnych dziurach,DOKTORANCKIE,FIZYKI,2019-06-11 00:00:00.000,"3,0",1563,"3,5"
4,Sily odsrodkowe w autobusach,LICENCJACKIE,FIZYKI,2021-06-21 00:00:00.000,"3,0",256,"3,5"
2,Opieka nad pacjentami z choroba Crohna,LICENCJACKIE,NAUK O ZDROWIU,2021-03-12 00:00:00.000,"3,0",35672,"3,5"
5,Inflacja w krajach Europy Srodkowo-Wschodniej,LICENCJACKIE,EKONOMII I ADMINISTRACJI,2018-12-21 00:00:00.000,"3,0",413,"3,5"
2,Opieka nad pacjentami z choroba Crohna,LICENCJACKIE,NAUK O ZDROWIU,2021-03-12 00:00:00.000,"3,0",8747,"4,5"


In [45]:
--procedura wstawiania pracy przez studenta
CREATE PROCEDURE sp_DodawaniePracyPrzezStudenta @p_idstudent char(10), @p_idstudentstudia int, @p_tytul varchar(100), @p_liczbaRobotnikow tinyint, @p_wydzial varchar(100), @p_grupaRodzaj varchar(50)
AS
BEGIN
DECLARE @v_idpraca int;
DECLARE @v_ileJuzWpisal int;
BEGIN TRY
    BEGIN TRANSACTION WPROWADZANIEPRACY
    --zabezpieczenie przed dodawaniem prac bez limitu przez złośliwego studenta (buffer overflow)
    SELECT @v_ileJuzWpisal = COUNT(*) FROM StudenciPrace WHERE (id_student = @p_idstudent AND id_student_studia = @p_idstudentstudia)
    IF @v_ileJuzWpisal > 4
    BEGIN;
        THROW 51000, 'Blad', 1
        ROLLBACK TRANSACTION WPROWADZANIEPRACY
    END
    INSERT INTO Prace (tytul,liczbaRobotnikow,wydzial,grupaRodzaj) VALUES (
        @p_tytul,
        @p_liczbaRobotnikow,
        @p_wydzial,
        @p_grupaRodzaj
    )
    SELECT @v_idpraca = SCOPE_IDENTITY()
    INSERT INTO StudenciPrace (id_student,id_praca,id_student_studia) VALUES (
        @p_idstudent,
        @v_idpraca,
        @p_idstudentstudia
    )
    COMMIT TRANSACTION WPROWADZANIEPRACY
    SELECT Concat('Twój numer pracy to: ',@v_idpraca,'. Przekaż go swoim współautorom aby mogli dodać się jako współautorzy')
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania pracy' AS 'BLAD'
    ROLLBACK TRANSACTION WPROWADZANIEPRACY
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.058

In [46]:
--widok dla zarządzającego na prace z przydzielonymi i nieprzydzielonymi promotorami
CREATE VIEW vw_Zarzadzajacy_PrzydzialPromotorowISzefow AS
SELECT
    Prace.id_praca, Prace.tytul, Prace.id_promotor, Prace.id_szefObrony, Prace.dataObrony,
    StudenciPrace.id_student
FROM Prace
INNER JOIN StudenciPrace ON Prace.id_praca = StudenciPrace.id_praca;

Commands completed successfully.

Total execution time: 00:00:00.051

In [47]:
--procedura dodawania sie jako wspolautor
CREATE PROCEDURE sp_DodawanieSieJakoWspolautor @p_idstudent char(10), @p_idstudentstudia int, @p_idpraca int 
AS
BEGIN
DECLARE @v_ileRobotnikow tinyint;
DECLARE @v_ileJuzSieDodalo tinyint;
BEGIN TRY
    BEGIN TRANSACTION DODAJSIEJAKOWSPOLAUTOR
    SELECT @v_ileJuzSieDodalo = COUNT(*) FROM StudenciPrace WHERE id_praca = @p_idpraca
    SELECT @v_ileRobotnikow = liczbaRobotnikow FROM Prace WHERE id_praca = @p_idpraca
    IF @v_ileRobotnikow <= @v_ileJuzSieDodalo
    BEGIN;
        THROW 51000, 'Blad', 1
        ROLLBACK TRANSACTION DODAJSIEJAKOWSPOLAUTOR
    END
    INSERT INTO StudenciPrace (id_student,id_praca,id_student_studia) VALUES (
        @p_idstudent,
        @p_idpraca,
        @p_idstudentstudia
    )
    COMMIT TRANSACTION DODAJSIEJAKOWSPOLAUTOR
    SELECT 'Poprawnie dodano studenta jako wspolautora'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania wspolautora'
    ROLLBACK TRANSACTION DODAJSIEJAKOWSPOLAUTOR
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.048

In [48]:
--procedura dodawania słów kluczowych
CREATE PROCEDURE sp_DodawanieSlowKluczowych @p_idpraca int, @slowo varchar(100)
AS
BEGIN
BEGIN TRY
    BEGIN TRANSACTION DODAJSLOWO
    INSERT INTO PraceSlowa VALUES (
        @p_idpraca,
        @slowo
    )
    COMMIT TRANSACTION DODAJSLOWO
    SELECT 'Poprawnie dodano slowo kluczowe'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania slowa kluczowego do pracy'
    ROLLBACK TRANSACTION DODAJSLOWO
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.049

In [49]:
--procedura przydzialu promotora przez zarządzającego
CREATE PROCEDURE sp_PrzydzielaniePromotora @p_idpraca int, @p_idpromotor int
AS 
BEGIN 
BEGIN TRY
    BEGIN TRANSACTION PRZYDZIALPROMOTORA
    UPDATE Prace SET id_promotor = @p_idpromotor WHERE id_praca = @p_idpraca
    COMMIT TRANSACTION PRZYDZIALPROMOTORA
    SELECT 'Poprawnie przydzielono promotora'
END TRY
BEGIN CATCH
    SELECT 'Wystapił błąd przydzielania promotora'
    ROLLBACK TRANSACTION PRZYDZIALPROMOTORA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.050

In [50]:
--procedura dodawania recenzenta przez promotora
CREATE PROCEDURE sp_DodawanieRecenzentaDoPracy @p_idpraca int, @p_idrecenzent int 
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION DODAWANIERECENZENTA
    INSERT INTO PraceRecenzenci (id_praca,id_recenzent) VALUES (
        @p_idpraca,
        @p_idrecenzent
    )
    COMMIT TRANSACTION DODAWANIERECENZENTA
    SELECT 'Poprawnie dodano recenzenta do zrecenzowania niezrecenzowanej przez niego pracy'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania recenzenta'
    ROLLBACK TRANSACTION DODAWANIERECENZENTA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.051

In [51]:
--procedura oceniania pracy przez recenzenta
CREATE PROCEDURE sp_OcenianiePracyPrzezRecenzenta @p_idpraca int, @p_idrecenzent int, @p_ocena decimal(2,1) 
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE PraceRecenzenci SET ocena = @p_ocena WHERE (id_praca = @p_idpraca AND id_recenzent = @p_idrecenzent)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano ocene'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania oceny'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.047

In [52]:
--procedura oceniania pracy przez promotor
CREATE PROCEDURE sp_OcenianiePracyPrzezPromotora @p_idpraca int, @p_ocena decimal(2,1) 
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE Prace SET ocenaPromotora = @p_ocena WHERE (id_praca = @p_idpraca)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano ocene'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania oceny'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.047

In [53]:
--procedura dodawania daty obrony i szefa obrony przez zarządzającego
CREATE PROCEDURE sp_DodajDateISzefaObrony @p_data datetime, @p_idpraca int, @p_idszefobrony int
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE Prace SET dataObrony = @p_data WHERE (id_praca = @p_idpraca)
    UPDATE Prace SET id_szefObrony = @p_idszefobrony WHERE (id_praca = @p_idpraca)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano date i szefa obrony'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania daty i szefa obrony'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.048

In [54]:
--procedura oceniania studenta przez szefa obrony
CREATE PROCEDURE sp_OcenianieStudenta @p_idpraca int, @p_idstudent char(10), @p_ocena decimal(2,1) 
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE StudenciPrace SET ocena = @p_ocena WHERE (id_praca = @p_idpraca AND id_student = @p_idstudent)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano ocene'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania oceny'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.048

In [55]:
--procedura wprowadzająca ocenę końcową pracy przez zarządzającego
CREATE PROCEDURE sp_WstawianieOcenyKoncowej @p_idpraca int, @p_ocena decimal(2,1) 
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE Prace SET ocenaKoncowa = @p_ocena WHERE (id_praca = @p_idpraca)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano ocene'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania oceny koncowej'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.047

In [56]:
--procedura dodajaca/updatujaca haslo przez pracownika
CREATE PROCEDURE sp_DodawanieHaslaPracownik @p_idpracownik int, @p_hashhaslo varchar(256)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE Pracownicy SET hash_haslo = @p_hashhaslo WHERE id_pracownik = @p_idpracownik
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano haslo'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania hasla'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.049

In [57]:
--procedura dodajaca/updatujaca haslo przez studenta
CREATE PROCEDURE sp_DodawanieHaslaStudent @p_idstudent char(10), @p_hashhaslo varchar(256)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    UPDATE Studenci SET hash_haslo = @p_hashhaslo WHERE id_student = @p_idstudent
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano haslo'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad dodawania hasla'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.048

In [58]:
--procedura konserwująca: usuwa prace bez promotora dodanego w ciągu 7 dni od dodania pracy
CREATE PROCEDURE sp_Konserwacja 
AS 
BEGIN
DECLARE @currDate datetime;
DECLARE @v_dataDodania datetime;
DECLARE @diff int;
DECLARE @v_idpraca int;
SET @currDate = GETDATE();
BEGIN TRY
    BEGIN TRANSACTION AAA
    DECLARE cur CURSOR LOCAL FOR SELECT id_praca, dataDodania FROM Prace WHERE id_promotor = 1
    OPEN cur
    FETCH NEXT FROM cur INTO @v_idpraca, @v_dataDodania
    WHILE @@FETCH_STATUS = 0
    BEGIN
    SELECT @diff = DATEDIFF(day, @v_dataDodania, @currDate)
    IF @diff > 7
    BEGIN
        DELETE FROM StudenciPrace WHERE id_praca = @v_idpraca
        DELETE FROM PraceSlowa WHERE id_praca = @v_idpraca
        --na wszelki wypadek, choć jest to prawie niemożliwe
        DELETE FROM PraceRecenzenci WHERE id_praca = @v_idpraca
        --i ostatecznie
        DELETE FROM Prace WHERE id_praca = @v_idpraca
    END
    FETCH  NEXT FROM cur INTO @v_idpraca, @v_dataDodania
    END
    CLOSE cur
    DEALLOCATE cur
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie zakonserwowano'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad usuwania prac bez promotora'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.049

In [59]:
EXEC sp_Konserwacja

(1 row affected)

Total execution time: 00:00:00.051

(No column name)
Poprawnie zakonserwowano


In [60]:
--procedura dodająca studenta
CREATE PROCEDURE sp_DodajStudenta @p_idstudent char(10), @p_imie varchar(20), @p_nazwisko varchar(20), @p_hashhaslo varchar(256)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Studenci VALUES (@p_idstudent, @p_imie, @p_nazwisko, @p_hashhaslo)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano studenta'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu studenta'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.048

In [61]:
--procedura dodająca studenta do studiow
CREATE PROCEDURE sp_PrzypiszStudentaDoStudiow @p_idstudent char(10), @p_idstudia int
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO StudenciStudia (id_student, id_studia) VALUES (@p_idstudent, @p_idstudia)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano studenta do studiow'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu studenta do studiow'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.051

In [62]:
--procedura dodająca rodzaje
CREATE PROCEDURE sp_DodajRodzaj @p_rodzaj varchar(50)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Rodzaje VALUES (@p_rodzaj)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano rodzaj studiow'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu rodzaju studiow'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.043

In [63]:
--procedura dodająca wydzialy
CREATE PROCEDURE sp_DodajWydzial @p_wydzial varchar(100)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Wydzialy VALUES (@p_wydzial)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano wydzial'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu wydzialu'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.043

In [64]:
--procedura dodająca studia (kierunki)
CREATE PROCEDURE sp_DodajStudia @p_wydzial varchar(100), @p_kierunek varchar(100), @p_rodzaj varchar(50)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Studia (wydzial, kierunek, rodzaj) VALUES (@p_wydzial, @p_kierunek, @p_rodzaj)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano studia'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu studiow'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.045

In [65]:
--procedura dodająca slowa kluczowe
CREATE PROCEDURE sp_DodajSlowo @p_slowo varchar(100)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Slowa VALUES (@p_slowo)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano slowo kluczowe'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu slowa kluczowego'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.046

In [66]:
--procedura dodająca stopnie
CREATE PROCEDURE sp_DodajStopnie @p_stopien varchar(30)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Stopnie VALUES (@p_stopien)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano stopien'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu stopnia'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.046

In [67]:
--procedura dodająca pracownika
CREATE PROCEDURE sp_DodajPracownika @p_imie varchar(20), @p_nazwisko varchar(20), @p_stopien varchar(30), @p_hashhaslo varchar(256)
AS 
BEGIN
BEGIN TRY
    BEGIN TRANSACTION AAA
    INSERT INTO Pracownicy (imie, nazwisko, stopien, hash_haslo) VALUES (@p_imie, @p_nazwisko, @p_stopien, @p_hashhaslo)
    COMMIT TRANSACTION AAA
    SELECT 'Poprawnie dodano pracownika'
END TRY
BEGIN CATCH
    SELECT 'Wystapil blad w dodawaniu pracownika'
    ROLLBACK TRANSACTION AAA
END CATCH;
END

Commands completed successfully.

Total execution time: 00:00:00.051

In [68]:
--tworzenie userów i nadawanie uprawnień. Hasła tylko dodane poglądowo./
--UWAGA: tych loginów i haseł nie używają indywidualni ludzie, lecz są używane przez serwer pośredniczący (np. HTTPS) między/
--właściwymi użytkownikami, a serwerem bazy danych.
CREATE USER Administrator WITH PASSWORD = '!1zzzz2@';
CREATE USER Student WITH PASSWORD = '!1yyyy2@';
CREATE USER Promotor WITH PASSWORD = '!1xxxx2@';
CREATE USER Recenzent WITH PASSWORD = '!1wwww2@';
CREATE USER Podgladacz WITH PASSWORD = '!1vvvv2@';
CREATE USER Historyk WITH PASSWORD = '!1uuuu2@';
CREATE USER Zarzadzajacy WITH PASSWORD = '!1tttt2@';
CREATE USER Szefobrony WITH PASSWORD = '!1ssss2@';
CREATE USER Wprowadzaczsystemu WITH PASSWORD = '!1rrrr2@';
CREATE USER Pracownik WITH PASSWORD = '!1pppp2@';
CREATE USER Aktualizator WITH PASSWORD = '!1qqqq2@';

GRANT SELECT, INSERT, UPDATE, DELETE TO Administrator;
GRANT EXECUTE ON sp_PrzypiszStudentaDoStudiow TO Administrator;
GRANT EXECUTE ON sp_DodajStudenta TO Administrator;
GRANT EXECUTE ON sp_DodajRodzaj TO Administrator;
GRANT EXECUTE ON sp_DodajWydzial TO Administrator;
GRANT EXECUTE ON sp_DodajStudia TO Administrator;
GRANT EXECUTE ON sp_DodajSlowo TO Administrator;
GRANT EXECUTE ON sp_DodajStopnie TO Administrator;
GRANT EXECUTE ON sp_DodajPracownika TO Administrator;
GRANT EXECUTE ON sp_SzukajPracePoDacie TO Administrator;
GRANT EXECUTE ON sp_SzukajPracePoKierunkuLubRoku TO Administrator;
GRANT EXECUTE ON sp_SzukajPracePoPromotorze TO Administrator;
GRANT EXECUTE ON sp_SzukajPracPoStudencie TO Administrator;
GRANT EXECUTE ON sp_SzukajPraceRecenzowane TO Administrator;
GRANT EXECUTE ON sp_SzukajPracePoTematyce TO Administrator;
GRANT EXECUTE ON sp_SzukajPracePoRodzaju TO Administrator;
GRANT EXECUTE ON sp_DodawaniePracyPrzezStudenta TO Administrator;
GRANT EXECUTE ON sp_DodawanieSieJakoWspolautor TO Administrator;
GRANT EXECUTE ON sp_DodawanieSlowKluczowych TO Administrator;
GRANT EXECUTE ON sp_DodawanieRecenzentaDoPracy TO Administrator;
GRANT EXECUTE ON sp_OcenianiePracyPrzezPromotora TO Administrator;
GRANT EXECUTE ON sp_OcenianiePracyPrzezRecenzenta TO Administrator;
GRANT EXECUTE ON sp_DodajDateISzefaObrony TO Administrator;
GRANT EXECUTE ON sp_PrzydzielaniePromotora TO Administrator;
GRANT EXECUTE ON sp_WstawianieOcenyKoncowej TO Administrator;
GRANT EXECUTE ON sp_OcenianieStudenta TO Administrator;
GRANT EXECUTE ON sp_DodawanieHaslaStudent TO Administrator;
GRANT EXECUTE ON sp_DodawanieHaslaPracownik TO Administrator;
GRANT EXECUTE ON sp_Konserwacja TO Administrator;

GRANT EXECUTE ON sp_SzukajPracPoStudencie TO Student;
GRANT EXECUTE ON sp_DodawaniePracyPrzezStudenta TO Student;
GRANT EXECUTE ON sp_DodawanieSieJakoWspolautor TO Student;
GRANT EXECUTE ON sp_DodawanieSlowKluczowych TO Student;
GRANT SELECT ON Studenci TO Student;

GRANT SELECT ON Pracownicy TO Pracownik;

GRANT EXECUTE ON sp_SzukajPracePoPromotorze TO Promotor;
GRANT EXECUTE ON sp_DodawanieRecenzentaDoPracy TO Promotor;
GRANT EXECUTE ON sp_OcenianiePracyPrzezPromotora TO Promotor;

GRANT SELECT ON vw_Recenzent_RecenzjePrac TO Recenzent;
GRANT EXECUTE ON sp_OcenianiePracyPrzezRecenzenta TO Recenzent;

GRANT SELECT ON vw_Zarzadzajacy_PrzydzialPromotorowISzefow TO Zarzadzajacy;
GRANT EXECUTE ON sp_DodajDateISzefaObrony TO Zarzadzajacy;
GRANT EXECUTE ON sp_PrzydzielaniePromotora TO Zarzadzajacy;
GRANT EXECUTE ON sp_WstawianieOcenyKoncowej TO Zarzadzajacy;

GRANT SELECT ON vw_SzefObrony_ObronyPrac TO Szefobrony;
GRANT EXECUTE ON sp_OcenianieStudenta TO Szefobrony;

GRANT EXECUTE ON sp_SzukajPracePoDacie TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPracePoKierunkuLubRoku TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPracePoPromotorze TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPracPoStudencie TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPraceRecenzowane TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPracePoTematyce TO Podgladacz;
GRANT EXECUTE ON sp_SzukajPracePoRodzaju TO Podgladacz;
GRANT SELECT TO Podgladacz;

GRANT SELECT, INSERT, UPDATE TO Wprowadzaczsystemu;

GRANT SELECT, INSERT TO Historyk;

GRANT SELECT, INSERT, UPDATE TO Aktualizator;
GRANT EXECUTE ON sp_PrzypiszStudentaDoStudiow TO Aktualizator;
GRANT EXECUTE ON sp_DodajStudenta TO Aktualizator;
GRANT EXECUTE ON sp_DodajRodzaj TO Aktualizator;
GRANT EXECUTE ON sp_DodajWydzial TO Aktualizator;
GRANT EXECUTE ON sp_DodajStudia TO Aktualizator;
GRANT EXECUTE ON sp_DodajSlowo TO Aktualizator;
GRANT EXECUTE ON sp_DodajStopnie TO Aktualizator;
GRANT EXECUTE ON sp_DodajPracownika TO Aktualizator;

GRANT EXECUTE ON sp_DodawanieHaslaStudent TO Student;
GRANT EXECUTE ON sp_DodawanieHaslaPracownik TO Pracownik;


Commands completed successfully.

Total execution time: 00:00:01.044

In [69]:
--Realizacja innych zapytań z harmonogramu przedmiotu (zap. 7,8,13 zrealizowano powyżej)
--Zapytanie 1
SELECT id_student, imie, nazwisko FROM Studenci ORDER BY nazwisko;

(23 rows affected)

Total execution time: 00:00:00.048

id_student,imie,nazwisko
135,Anna,Andrzejewska
138,Bartosz,Benedyktynski
256,Czeslawa,Cymbalinska
413,Dionizy,Doglebny
1123,Eleonora,Elektryfikujaca
35672,Franciszek,Ferdynancki
1563,Franciszek,Frankowski
8747,Grzegorz,Gorycki
11234,Henryk,Heniawski
987,Ilona,Ilanicka


In [70]:
--Zapytanie 2
SELECT stopien,imie,nazwisko FROM Pracownicy ORDER BY nazwisko;

(22 rows affected)

Total execution time: 00:00:00.047

stopien,imie,nazwisko
DR INZ,Marek,Moczydlowski
PROF,Marek,Moczydlowski
MGR,ImiePracownik1,NazwiskoPracownik1
DR,ImiePracownik20,NazwiskoPracownik10
MGR,ImiePracownik21,NazwiskoPracownik11
MGR,ImiePracownik22,NazwiskoPracownik12
MGR,ImiePracownik23,NazwiskoPracownik13
LIC,ImiePracownik2,NazwiskoPracownik2
DR,ImiePracownik3,NazwiskoPracownik3
PROF,ImiePracownik4,NazwiskoPracownik4


In [71]:
--Zapytanie 3
SELECT tytul FROM Prace ORDER BY tytul;

(8 rows affected)

Total execution time: 00:00:00.047

tytul
Elektrochemia kardiomiocytów
Inflacja w krajach Europy Srodkowo-Wschodniej
Kwantowa teoria pola w czarnych dziurach
Nitrozaminy w komórkach ludzkich
Opieka nad pacjentami z choroba Crohna
Pomiary napiecia w procesorach x86
Sily odsrodkowe w autobusach
Zastosowania kryptografii na stronach rzadowych


In [72]:
--Zapytanie 4
SELECT tytul,dataObrony FROM prace ORDER BY dataObrony;

(8 rows affected)

Total execution time: 00:00:02.416

tytul,dataObrony
Inflacja w krajach Europy Srodkowo-Wschodniej,2018-12-21 00:00:00.000
Kwantowa teoria pola w czarnych dziurach,2019-06-11 00:00:00.000
Pomiary napiecia w procesorach x86,2020-01-10 00:00:00.000
Zastosowania kryptografii na stronach rzadowych,2020-01-10 00:00:00.000
Elektrochemia kardiomiocytów,2020-02-12 00:00:00.000
Nitrozaminy w komórkach ludzkich,2020-03-12 00:00:00.000
Opieka nad pacjentami z choroba Crohna,2021-03-12 00:00:00.000
Sily odsrodkowe w autobusach,2021-06-21 00:00:00.000


In [73]:
--Zapytanie 5
SELECT tytul,ocenaKoncowa,dataObrony FROM Prace ORDER BY ocenaKoncowa,dataObrony;


(8 rows affected)

Total execution time: 00:00:00.048

tytul,ocenaKoncowa,dataObrony
Inflacja w krajach Europy Srodkowo-Wschodniej,"3,0",2018-12-21 00:00:00.000
Kwantowa teoria pola w czarnych dziurach,"3,0",2019-06-11 00:00:00.000
Pomiary napiecia w procesorach x86,"3,0",2020-01-10 00:00:00.000
Zastosowania kryptografii na stronach rzadowych,"3,0",2020-01-10 00:00:00.000
Elektrochemia kardiomiocytów,"3,0",2020-02-12 00:00:00.000
Nitrozaminy w komórkach ludzkich,"3,0",2020-03-12 00:00:00.000
Opieka nad pacjentami z choroba Crohna,"3,0",2021-03-12 00:00:00.000
Sily odsrodkowe w autobusach,"3,0",2021-06-21 00:00:00.000


In [74]:
--Zapytanie 6
SELECT tytul, grupaRodzaj FROM Prace GROUP BY grupaRodzaj, tytul;

(8 rows affected)

Total execution time: 00:00:00.989

tytul,grupaRodzaj
Nitrozaminy w komórkach ludzkich,MAGISTERSKIE
Opieka nad pacjentami z choroba Crohna,LICENCJACKIE
Kwantowa teoria pola w czarnych dziurach,DOKTORANCKIE
Sily odsrodkowe w autobusach,LICENCJACKIE
Inflacja w krajach Europy Srodkowo-Wschodniej,LICENCJACKIE
Pomiary napiecia w procesorach x86,INZYNIERSKIE
Zastosowania kryptografii na stronach rzadowych,INZYNIERSKIE
Elektrochemia kardiomiocytów,MAGISTERSKIE


In [75]:
--Zapytania 7,8 było wcześniej; Zapytania 9,10,11
EXEC sp_SzukajPraceRecenzowane 0,'','';

Wyswietla prace recenzowane przez okreslonych pracowników uczelni

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.059

tytul,autorzy_i_ich_oceny,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Nitrozaminy w komórkach ludzkich,Bartosz Benedyktynski 4.0;,PROF Rakoczy Rondecki,"DR Zygfryd Zebrzanski, ocena: 5.0;","3,0",2020-03-12 00:00:00.000,CHEMII,MAGISTERSKIE,KONSTRUKCJE ZELBETOWE; NITROZAMINY; PRACA DYPLOMOWA;
Opieka nad pacjentami z choroba Crohna,Eleonora Elektryfikujaca 3.5;Franciszek Ferdynancki 3.5;Grzegorz Gorycki 4.5;,DR Zygfryd Zebrzanski,"PROF Rakoczy Rondecki, ocena: 4.0;DR HAB Paulina Penderecka, ocena: 3.0;","3,0",2021-03-12 00:00:00.000,NAUK O ZDROWIU,LICENCJACKIE,NITROZAMINY; PRACA DYPLOMOWA;
Kwantowa teoria pola w czarnych dziurach,Henryk Heniawski 3.5;Franciszek Frankowski 3.5;,PROF Rakoczy Rondecki,"DR HAB Paulina Penderecka, ocena: 5.0;","4,0",2019-06-11 00:00:00.000,FIZYKI,DOKTORANCKIE,NEUROLEPTYKI; PRACA DYPLOMOWA;
Sily odsrodkowe w autobusach,Czeslawa Cymbalinska 3.5;,DR HAB Paulina Penderecka,"PROF Rakoczy Rondecki, ocena: 5.0;MGR Tadeusz Tokarski, ocena: 4.5;INZ Ola Oskarska, ocena: 3.5;","5,0",2021-06-21 00:00:00.000,FIZYKI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; PRACA DYPLOMOWA;
Inflacja w krajach Europy Srodkowo-Wschodniej,Dionizy Doglebny 3.5;,DR INZ Marek Moczydlowski,"PROF Rakoczy Rondecki, ocena: 3.5;","4,5",2018-12-21 00:00:00.000,EKONOMII I ADMINISTRACJI,LICENCJACKIE,NOSOROZCE NA SAWANNIE; POTENCJAL BLONOWY; PRACA DYPLOMOWA;
Pomiary napiecia w procesorach x86,Bartosz Benedyktynski 3.5;,PROF Rakoczy Rondecki,"MGR Tadeusz Tokarski, ocena: 4.5;","3,5",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;
Zastosowania kryptografii na stronach rzadowych,Anna Andrzejewska 5.0;,DR INZ Marek Moczydlowski,"MGR Tadeusz Tokarski, ocena: 3.5;","3,0",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;
Elektrochemia kardiomiocytów,Ilona Ilanicka 4.0;,PROF Marek Moczydlowski,"LIC Sonia Szarecka, ocena: 3.5;DR INZ Marek Moczydlowski, ocena: 4.5;","5,0",2020-02-12 00:00:00.000,CHEMII,MAGISTERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [76]:
--Zapytanie 12
EXEC sp_SzukajPracPoStudencie '','Benedyktyński','Bartosz';

Wyswietla prace studentów o imionach i nazwiskach: Bartosz Benedyktynski

(1 row affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.056

tytul,wspolautorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,ocena_koncowa,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Nitrozaminy w komórkach ludzkich,BRAK,PROF Rakoczy Rondecki,"DR Zygfryd Zebrzanski, ocena: 5.0;","3,0","4,0",2020-03-12 00:00:00.000,CHEMII,MAGISTERSKIE,KONSTRUKCJE ZELBETOWE; NITROZAMINY; PRACA DYPLOMOWA;
Pomiary napiecia w procesorach x86,BRAK,PROF Rakoczy Rondecki,"MGR Tadeusz Tokarski, ocena: 4.5;","3,5","3,5",2020-01-10 00:00:00.000,ELEKTRYCZNY,INZYNIERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;


In [77]:
--Zapytanie 13 było uprzednio; Zapytanie 14
EXEC sp_SzukajPracePoRodzaju 'MAGISTERSKIE';

Wyswietla prace okreslonego rodzaju

(1 row affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.048

tytul,autorzy,promotor,recenzenci_i_ich_oceny,ocena_promotora,data_obrony,wydzial,rodzaj_pracy,slowa_kluczowe
Nitrozaminy w komórkach ludzkich,Bartosz Benedyktynski;,PROF Rakoczy Rondecki,"DR Zygfryd Zebrzanski, ocena: 5.0;","3,0",2020-03-12 00:00:00.000,CHEMII,MAGISTERSKIE,KONSTRUKCJE ZELBETOWE; NITROZAMINY; PRACA DYPLOMOWA;
Elektrochemia kardiomiocytów,Ilona Ilanicka;,PROF Marek Moczydlowski,"LIC Sonia Szarecka, ocena: 3.5;DR INZ Marek Moczydlowski, ocena: 4.5;","5,0",2020-02-12 00:00:00.000,CHEMII,MAGISTERSKIE,PRACA DYPLOMOWA; PRAD ELEKTRYCZNY;
